In [1]:
# Get raw data
with open('input/22.txt', 'r') as f:
    rawinput = f.read().strip()

In [2]:
import numpy as np

In [3]:
# Part 1
depth, targ_x, targ_y = [int(j) for i in rawinput.split('\n') for j in i.split(':')[1].split(',')]
# depth, targ_x, targ_y = 510,10,10

x = np.expand_dims(np.arange(targ_x+1), 0)
y = np.expand_dims(np.arange(targ_y+1), 1)
geoindex = np.where((x==0)|(y==0), x*16807 + y*48271, -1)
geoindex[targ_y, targ_x] = 0
while True:
    erosion = np.where(geoindex < 0, -1, (geoindex+depth)%20183)
    geoindex[1:, 1:] = np.where((geoindex[1:, 1:] >= 0) 
                                | ((en:=erosion[:-1,1:])<0) 
                                | ((ew:=erosion[1:,:-1])<0),
                                geoindex[1:, 1:], 
                                en*ew)
    if np.all(erosion != -1):
        break

np.sum(np.mod(erosion,3)).item()

11575

In [4]:
# Part 2
class Spelunk(object):
    def __init__(self, depth, target_loc):
        self.depth = depth
        self.target_loc = target_loc
        self.do_mapping()
        
    def do_mapping(self, pad=100):
        y = np.expand_dims(np.arange(self.target_loc[0]+1+pad), 1)
        x = np.expand_dims(np.arange(self.target_loc[1]+1+pad), 0)
        geoindex = np.where((x==0)|(y==0), x*16807 + y*48271, -1)
        geoindex[*self.target_loc] = 0
        while True:
            erosion = np.where(geoindex < 0, -1, (geoindex+depth)%20183)
            geoindex[1:, 1:] = np.where((geoindex[1:, 1:] >= 0) 
                                        | ((en:=erosion[:-1,1:])<0) 
                                        | ((ew:=erosion[1:,:-1])<0),
                                        geoindex[1:, 1:], 
                                        en*ew)
            if np.all(erosion != -1):
                break
        self.reg_type = np.mod(erosion, 3)
        
    def find_path(self):
        loc = np.array([[0,0]])
        kit = np.array([1])
        time = np.array([0])
        mvhist = np.empty((1,0), dtype='<U1')
        min_time = -np.ones((*self.reg_type.shape,3), dtype=int)
        min_time[0,0,1] = 0
        pcd = 1000
        while pcd:
            loc = np.repeat(loc, 6, axis=0) + np.tile([[-1,0], [1,0], [0,-1], [0,1], [0,0], [0,0]], (loc.shape[0],1))
            kit = np.mod(np.repeat(kit, 6, axis=0) + np.tile([0,0,0,0,1,2], kit.shape[0]), 3)
            time = np.repeat(time, 6, axis=0) + np.tile([1,1,1,1,7,7], time.shape[0])
            mvhist = np.column_stack((np.repeat(mvhist, 6, axis=0), np.tile(list('NSWE12'), mvhist.shape[0])))
        
            filt = (np.all(loc >= 0, axis=1)
                    & np.all(loc < self.reg_type.shape, axis=1))
            loc, kit, time, mvhist = [i[filt] for i in [loc, kit, time, mvhist]]

            filt = ((self.reg_type[loc[:,0], loc[:,1]] != kit)
                    & (((z:=min_time[loc[:,0], loc[:,1], kit]) < 0) | (time < z)))
            loc, kit, time, mvhist = [i[filt] for i in [loc, kit, time, mvhist]]
        
            filt = np.any((z:=(y:=np.column_stack((loc, kit, time)))
                            [(sortix:=np.lexsort([y[:,i] for i in range(y.shape[1]-1,-1,-1)]))])
                        != np.append(-np.ones_like(z[:1]), z[:-1], axis=0), 
                        axis=1)[np.argsort(sortix)]
            loc, kit, time, mvhist = [i[filt] for i in [loc, kit, time, mvhist]]
        
            min_time[loc[:,0], loc[:,1], kit] = time
            
            if (np.all(np.subtract(min_time.shape[:2], self.target_loc) > 0) 
                and (z:=min_time[*self.target_loc, 1]) > 0):
                pcd -= 1
        return z.item()

self = Spelunk(depth, (targ_y,targ_x))
self.find_path()

1068